In [113]:
import pandas as pd

In [114]:
df = pd.read_csv("spotify_millsongdata.csv")

In [115]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [116]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [117]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [118]:
df.shape

(5000, 3)

In [119]:
df.head()

,artist,song,text
0,Doris Day,Everybody Loves A Lover,"Everybody loves a lover \r\nI'm a lover, ever..."
1,Bruce Springsteen,Dancing In The Dark,"I get up in the evening, and I ain't got nothi..."
2,Wilson Phillips,Fueled For Houston,Fueled for Houston \r\nI missed all my calls ...
3,Loretta Lynn,Act Naturally,They're gonna put me in the movies they're gon...
4,Zao,All Else Failed,(A throne in Heaven sat empty for 33 years.) W...


In [120]:
df['text'][0]

"Everybody loves a lover  \r\nI'm a lover, everybody loves me  \r\nAnyhow, that's how I feel  \r\nWow, I feel just like a Pollyanna  \r\n  \r\nI should worry, not for nothin'  \r\nEverybody loves me, yes they do  \r\nAnd I love everybody  \r\nSince I fell in love with you  \r\n  \r\nWho's the most popular personality?  \r\nI can't help thinkin' it's no one else but me  \r\nGee, I feel just about ten feet tall, havin' a ball  \r\nGuess ya might call me a Pollyanna  \r\n  \r\nEverybody loves a lover  \r\n  \r\nI should worry, not for nothin'  \r\nEverybody loves me, yes they do  \r\nAnd I love everybody  \r\nSince I fell in love with  \r\nFell in love with  \r\nFell in love with you...  \r\n  \r\n(Call me a Pollyanna, do)\r\n\r\n"

In [121]:
df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

0       everybody loves a lover  \r i'm a lover, every...
1       i get up in the evening, and i ain't got nothi...
2       fueled for houston  \r i missed all my calls t...
3       they're gonna put me in the movies they're gon...
4       (a throne in heaven sat empty for 33 years.) w...
                              ...                        
4995    shadows falling in the noonday sun  \r blue fe...
4996    so many things i think about  \r when i look f...
4997    if there were no dreams  \r and there were no ...
4998    to thomas s.eiselberg, a very rich man, who wa...
4999    i need a woman 'bout twice my age  \r a lady o...
Name: text, Length: 5000, dtype: object

In [122]:
df['text'][0]

"Everybody loves a lover  \r\nI'm a lover, everybody loves me  \r\nAnyhow, that's how I feel  \r\nWow, I feel just like a Pollyanna  \r\n  \r\nI should worry, not for nothin'  \r\nEverybody loves me, yes they do  \r\nAnd I love everybody  \r\nSince I fell in love with you  \r\n  \r\nWho's the most popular personality?  \r\nI can't help thinkin' it's no one else but me  \r\nGee, I feel just about ten feet tall, havin' a ball  \r\nGuess ya might call me a Pollyanna  \r\n  \r\nEverybody loves a lover  \r\n  \r\nI should worry, not for nothin'  \r\nEverybody loves me, yes they do  \r\nAnd I love everybody  \r\nSince I fell in love with  \r\nFell in love with  \r\nFell in love with you...  \r\n  \r\n(Call me a Pollyanna, do)\r\n\r\n"

In [123]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [124]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [125]:
df['text'].apply(lambda x: tokenization(x))

0       everybodi love a lover i 'm a lover , everybod...
1       i get up in the even , and i ai n't got noth t...
2       fuel for houston i miss all my call thi morn t...
3       they 're gon na put me in the movi they 're go...
4       ( a throne in heaven sat empti for 33 year . )...
                              ...                        
4995    shadow fall in the noonday sun blue feel to th...
4996    so mani thing i think about when i look far aw...
4997    if there were no dream and there were no dream...
4998    to thoma s.eiselberg , a veri rich man , who w...
4999    i need a woman 'bout twice my age a ladi of no...
Name: text, Length: 5000, dtype: object

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [127]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [128]:
similarity[0]

array([1.00000000e+00, 2.71186564e-02, 1.10239599e-02, ...,
       3.54979680e-02, 9.66973653e-04, 6.48364629e-03])

In [130]:
df[df['song'] == 'Dancing In The Dark'].index[0]

1

In [131]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:5]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [132]:
recommendation('Dancing In The Dark')

["Just Can't Get Enough", 'Dancing On The Ceiling', 'Just Fine', 'In The Dark']